# 1

In [ ]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install datasets

In [ ]:
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.4 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.5 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub


In [ ]:

!pip install accelerate bitsandbytes

In [ ]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
import os
import json

In [ ]:
from huggingface_hub import login

login(token="")

In [ ]:
vector_store_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/medical_rag_vectorstore_2k"
dataset_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/medqu.jsonl"
base_model_id = "meta-llama/Llama-2-7b-hf"
adapter_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.3 fine tune/medical_lora_adapter"

In [ ]:
class MedicalRAG:
    def __init__(self, base_model_id, adapter_path, vector_store_path):
        print("Loading models and vector store...")

        # Load embeddings model
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Check if vector store exists, create it if it doesn't
        if not os.path.exists(os.path.join(vector_store_path, "index.faiss")):
            print(f"Vector store not found at {vector_store_path}. Creating new vector store...")
            self._create_vector_store(vector_store_path)

        # Load the vector store
        self.vector_store = FAISS.load_local(vector_store_path, self.embeddings,allow_dangerous_deserialization=True)
        self.retriever = self.vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_id)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load fine-tuned model
        self.base_model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            device_map="auto",
            torch_dtype=torch.float16,
        load_in_4bit=True

        )
        self.model = PeftModel.from_pretrained(self.base_model, adapter_path)
        print("Models and vector store loaded successfully!")

    def _create_vector_store(self, vector_store_path):
        # Ensure directory exists
        os.makedirs(vector_store_path, exist_ok=True)

        # Load the dataset
        with open(dataset_path, 'r') as f:
            medical_data = [json.loads(line) for line in f]

        # Use the first 2000 documents (same subset used for fine-tuning)
        subset_data = medical_data[:2000]
        print(f"Using {len(subset_data)} documents for RAG knowledge base")

        # Extract text from the subset
        documents = []
        for item in subset_data:
            text = item.get('text', '')
            documents.append(text)

        # Split into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        chunks = text_splitter.create_documents(documents)
        print(f"Created {len(chunks)} text chunks for embedding")

        # Create vector store
        vector_store = FAISS.from_documents(chunks, self.embeddings)
        vector_store.save_local(vector_store_path)
        print(f"Vector store created and saved to {vector_store_path}")

    def answer_question(self, query):
        # Retrieve relevant documents
        docs = self.retriever.get_relevant_documents(query)
        context = "\n\n".join([doc.page_content for doc in docs])

        # Create prompt with retrieved context
        prompt = f"""<s>[INST] <<SYS>>
Provide accurate, concise answers to medical questions based on the following information:

{context}
<</SYS>>

Question: {query} [/INST]"""

        # Generate response
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs.input_ids,
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9
            )
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response.split("[/INST]")[-1].strip()

In [ ]:
vector_store_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/medical_rag_vectorstore_2k"
dataset_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/medqu.jsonl"
base_model_id = "meta-llama/Llama-2-7b-hf"
adapter_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.3 fine tune/medical_lora_adapter"

In [ ]:
import pickle

# Create and save the MedicalRAG object
def setup_medical_rag():
    base_model_id = "meta-llama/Llama-2-7b-hf"
    adapter_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.3 fine tune/medical_lora_adapter"
    vector_store_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/medical_rag_vectorstore_2k"

    # Create the RAG system
    med_rag = MedicalRAG(base_model_id, adapter_path, vector_store_path)

    # Save the system state (except the model which is too large)
    # We'll just save the paths and reload when needed
    rag_config = {
        'base_model_id': base_model_id,
        'adapter_path': adapter_path,
        'vector_store_path': vector_store_path
    }

    with open('/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/med_rag_config.pkl', 'wb') as f:
        pickle.dump(rag_config, f)

    return med_rag

# Create the system once
med_rag = setup_medical_rag()

Loading models and vector store...


/tmp/ipython-input-2129357733.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Models and vector store loaded successfully!


In [ ]:
import pickle

# Load the configuration
with open('/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.5/med_rag_config.pkl', 'rb') as f:
    rag_config = pickle.load(f)

# Recreate the RAG system using saved configuration
med_rag = MedicalRAG(
    rag_config['base_model_id'],
    rag_config['adapter_path'],
    rag_config['vector_store_path']
)

#
query = "What causes multiple sclerosis?"
answer = med_rag.answer_question(query)
print(f"Query: {query}")
print(f"Answer: {answer}")

In [ ]:
# query = "answer this in short: when can I use panadol?"
answer = med_rag.answer_question(query)
print(f"Query: {query}")
print(f"Answer: {answer}")

# 12